In [22]:
!pip install linearmodels
!pip install statsmodels

In [23]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

In [88]:
data = pd.read_csv('Data_Regression.csv', delimiter=';')

data

,Year,Total Debt / Total Assets,ROCE,Current ratio,log (assets),Short Term Debt / Total Assets,Long Term Debt / Total Assets,COVID (Dummy),Company code
0,2018,"0,00 %","-50,00 %",NaN,"3,359836","0,00 %","0,00 %",0,1
1,2019,"0,00 %","-100,00 %",NaN,"3,050534","0,00 %","0,00 %",0,1
2,2020,"75,60 %","-173,62 %","67,64 %","5,030798","75,60 %","0,00 %",1,1
3,2021,"111,65 %","-667,87 %","97,40 %","6,079966","97,51 %","14,13 %",1,1
4,2022,"100,16 %","101,38 %","103,75 %","6,415391","94,02 %","6,15 %",1,1
...,...,...,...,...,...,...,...,...,...
240,2018,"56,33 %","-7,22 %","77,22 %","7,320354","40,19 %","16,14 %",0,49
241,2019,"53,05 %","-2,18 %","81,95 %","7,316813","40,40 %","12,65 %",0,49
242,2020,"46,02 %","-25,70 %","92,99 %","7,308805","43,43 %","2,59 %",1,49
243,2021,"71,15 %","-67,15 %","64,45 %","7,071486","42,56 %","28,59 %",1,49


In [89]:
data.columns

Index(['Year', 'Total Debt / Total Assets ', 'ROCE', 'Current ratio',
       'log (assets)', 'Short Term Debt / Total Assets',
       'Long Term Debt / Total Assets', 'COVID (Dummy)', 'Company code'],
      dtype='object')

In [78]:
for col in data.columns:
  null_rows = data[col].isnull()
  for index, is_null in null_rows.items():
    if is_null:
      print(f'None/NaN in column {col} at row {index}')

None/NaN in column Total Debt / Total Assets  at row 60
None/NaN in column Total Debt / Total Assets  at row 80
None/NaN in column Total Debt / Total Assets  at row 195
None/NaN in column Total Debt / Total Assets  at row 225
None/NaN in column ROCE at row 60
None/NaN in column ROCE at row 80
None/NaN in column ROCE at row 182
None/NaN in column ROCE at row 195
None/NaN in column ROCE at row 225
None/NaN in column Current ratio at row 0
None/NaN in column Current ratio at row 1
None/NaN in column Current ratio at row 60
None/NaN in column Current ratio at row 80
None/NaN in column Current ratio at row 195
None/NaN in column Current ratio at row 209
None/NaN in column Current ratio at row 225
None/NaN in column log (assets) at row 60
None/NaN in column log (assets) at row 80
None/NaN in column log (assets) at row 195
None/NaN in column log (assets) at row 225
None/NaN in column Short Term Debt / Total Assets at row 60
None/NaN in column Short Term Debt / Total Assets at row 80
None/NaN 

In [90]:
data['Total Debt / Total Assets '] = data['Total Debt / Total Assets '].astype(str).str.replace(',','.').str.rstrip('%').astype('float') / 100
data['ROCE'] = data['ROCE'].astype(str).str.replace(',','.').str.rstrip('%').astype('float') / 100
data['Current ratio'] = data['Current ratio'].astype(str).str.replace(',','.').str.rstrip('%').astype('float') / 100
data['log (assets)'] = data['log (assets)'].astype(str).str.replace(',','.').astype('float')
data['Short Term Debt / Total Assets'] = data['Short Term Debt / Total Assets'].astype(str).str.replace(',','.').str.rstrip('%').astype('float') / 100
data['Long Term Debt / Total Assets'] = data['Long Term Debt / Total Assets'].astype(str).str.replace(',','.').str.rstrip('%').astype('float') / 100


In [91]:
data = data.set_index(['Company code', 'Year'])

In [94]:
data

Total Debt / Total Assets     ROCE  Current ratio  \
Company code Year                                                      
1            2018                      0.0000 -0.5000            NaN   
             2019                      0.0000 -1.0000            NaN   
             2020                      0.7560 -1.7362         0.6764   
             2021                      1.1165 -6.6787         0.9740   
             2022                      1.0016  1.0138         1.0375   
...                                       ...     ...            ...   
49           2018                      0.5633 -0.0722         0.7722   
             2019                      0.5305 -0.0218         0.8195   
             2020                      0.4602 -0.2570         0.9299   
             2021                      0.7115 -0.6715         0.6445   
             2022                      0.6756 -0.0203         0.8625   

                   log (assets)  Short Term Debt / Total Assets  \
Company code Year                                                 
1            2018      3.359836                          0.0000   
             2019      3.050534                          0.0000   
             2020      5.030798                          0.7560   
             2021      6.079966                          0.9751   
             2022      6.415391                          0.9402   
...                         ...                             ...   
49           2018      7.320354                          0.4019   
             2019      7.316813                          0.4040   
             2020      7.308805                          0.4343   
             2021      7.071486                          0.4256   
             2022      7.019670                          0.2670   

                   Long Term Debt / Total Assets  COVID (Dummy)  
Company code Year                                                
1            2018                         0.0000              0  
             2019                         0.0000              0  
             2020                         0.0000              1  
             2021                         0.1413              1  
             2022                         0.0615              1  
...                                          ...            ...  
49           2018                         0.1614              0  
             2019                         0.1265              0  
             2020                         0.0259              1  
             2021                         0.2859              1  
             2022                         0.4087              1  

[245 rows x 7 columns]

# SHIT BELOW

In [93]:
dependent_vars = ['Total Debt / Total Assets ', 'Short Term Debt / Total Assets', 'Long Term Debt / Total Assets']
independent_vars = ['ROCE', 'Current ratio', 'log (assets)', 'COVID (Dummy)']

for dep_var in dependent_vars:
  print(dep_var)

  model = PanelOLS(data[dep_var], sm.add_constant(data[independent_vars]), entity_effects=False, time_effects=False, drop_absorbed=False)
  results = model.fit(cov_type='robust')

  print(f'Regression results for {dep_var}:')
  print(results, '\n')


Total Debt / Total Assets 
Regression results for Total Debt / Total Assets :
                              PanelOLS Estimation Summary                               
Dep. Variable:     Total Debt / Total Assets    R-squared:                        0.3022
Estimator:                           PanelOLS   R-squared (Between):              0.3857
No. Observations:                         237   R-squared (Within):               0.0352
Date:                        Wed, Dec 20 2023   R-squared (Overall):              0.3022
Time:                                11:31:14   Log-likelihood                    36.356
Cov. Estimator:                        Robust                                           
                                                F-statistic:                      25.118
Entities:                                  49   P-value                           0.0000
Avg Obs:                               4.8367   Distribution:                   F(4,232)
Min Obs:                        

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1214: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [39]:
data = data.drop('COVID (Dummy)', axis=1)

mean_values = data.mean()
median_values = data.median()
range_values = data.max() - data.min()
variance_values = data.var()
std_deviation_values = data.std()
skewness_values = data.skew()
kurtosis_values = data.kurt()

# Display Descriptive Statistics
print("\nDescriptive Statistics:")
print("Mean Values:\n", mean_values)
print("\nMedian Values:\n", median_values)
print("\nRange Values:\n", range_values)
print("\nVariance Values:\n", variance_values)
print("\nStandard Deviation Values:\n", std_deviation_values)
print("\nSkewness Values:\n", skewness_values)
print("\nKurtosis Values:\n", kurtosis_values)

# Minimum and Maximum Values
min_values = data.min()
max_values = data.max()

# Display Minimum and Maximum Values
print("\nMinimum Values:\n", min_values)
print("\nMaximum Values:\n", max_values)


Descriptive Statistics:
Mean Values:
 Total Debt / Total Assets         6.070286e-07
ROCE                              2.554121e-05
Current ratio                     1.781928e-04
log (assets)                      6.899013e+00
Short Term Debt / Total Assets    5.215079e-05
Long Term Debt / Total Assets     8.552199e-02
dtype: float64

Median Values:
 Total Debt / Total Assets         6.339000e-07
ROCE                              1.637000e-05
Current ratio                     1.226600e-04
log (assets)                      6.796935e+00
Short Term Debt / Total Assets    4.762000e-05
Long Term Debt / Total Assets     0.000000e+00
dtype: float64

Range Values:
 Total Debt / Total Assets         0.000002
ROCE                              0.001493
Current ratio                     0.002342
log (assets)                      5.879440
Short Term Debt / Total Assets    0.000113
Long Term Debt / Total Assets     0.801300
dtype: float64

Variance Values:
 Total Debt / Total Assets         6.468561

In [40]:
grouped_data = data.groupby(level=1)

# Define the summary statistics functions
summary_functions = ['mean', 'median', lambda x: x.max() - x.min(), 'var', 'std', 'skew', 'kurt']
# Compute statistics for each year
for year, group in grouped_data:
    print(f"\nYear {year} Descriptive Statistics:")
    summary_stats = group.agg(summary_functions)
    print(summary_stats)


Year 2018 Descriptive Statistics:
          Total Debt / Total Assets           ROCE  Current ratio  \
mean                    5.907089e-07  4.775800e-05   1.743061e-04   
median                  5.791000e-07  2.509000e-05   1.219500e-04   
<lambda>                1.134800e-06  5.563200e-04   6.949100e-04   
var                     6.999556e-14  9.882401e-09   1.772567e-08   
std                     2.645667e-07  9.941027e-05   1.331378e-04   
skew                    0.000000e+00  1.668404e+00   2.233532e+00   
kurt                    0.000000e+00  4.714648e+00   6.233999e+00   

          log (assets)  Short Term Debt / Total Assets  \
mean          6.813770                    5.135911e-05   
median        6.783034                    4.217000e-05   
<lambda>      5.474604                    1.134800e-04   
var           0.726709                    7.570065e-10   
std           0.852472                    2.751375e-05   
skew         -1.206363                    3.558881e-01   
kurt  